In [18]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import selenium
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import csv


def xpath(row=0,col=0,sub=0):
    xpath = '//app/city-history/city-history-layout/div/div[2]/'\
            + 'section/div[2]/div[3]/div/div/div/div/'\
            + 'city-history-observation/div/div[2]/table'\
            + '/tbody/tr/td[' + str(col+1) + ']'\
            + '/table/tbody/tr[' + str(row+1) + ']'\
            + '/td[' + str(sub+1) + ']'
    return xpath



def read_webpage(driver,url):
    # Record start time (for scraper performance purposes)
    start_time = datetime.now()
    
    driver.get(url)
    print('webpage gotten')
    time.sleep(3)
    
    webpage_data = []

    # header stuff
    print('header stuff')
    pass

    # rows
    row_idx = 3
    column_idx = 0
    subcolumn_idx = 0

    # Iterate through rows.
    while True:
        # For each row,
        # ...initialize column_idx to 0 and row_data to [], and ...
        column_idx = 0
        row_data = []
                
        print('row idx:', row_idx) ###########################################################
        try:
            # ...try to read the subcolumn at the following xpath to see if the row exists.
            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

            # If that worked, then iterate through columns.
            while True:
                # For each column,
                # ...initialize subcolumn_idx to 0 and column_data to [], and ...
                subcolumn_idx = 0
                column_data = []
                
                print('column idx:', column_idx) #############################################
                
                try:
                    # ...try to read the subcolumn at the following xpath to see if the column exists.
                    raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                    # If that worked, then iterate through subcolumns
                    while True:
                        # For each subcolumn,
                        try:
                            # ...try to read the subcolumn at the following xpath to see if it exists.
                            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                            # If that worked, then try to convert raw_datum.text to a float.
                            try:
                                if column_idx == 0:
                                    datum = int(raw_datum.text) # Store dates as integers.
                                else:
                                    datum = float(raw_datum.text) # Store measured values as floats.
                            except:
                                datum = float('nan') # Store dashes and anything that doesn't cooperate as NaNs.
                            column_data.append(datum)
                            print(datum,end='\t') #########################################
                        except:
                            # If it didn't work, there is no subcolumn at that index. Break subcolumn loop.
                            print('subcolumn',subcolumn_idx,'not found, end column ~') #####################################################
                            break

                        # next subcolumn
                        subcolumn_idx += 1

                    # That's it for the subcolumns in this column. Store column_data in row_data.
                    print('column data',column_data)
                    row_data.extend(column_data)

                except:
                    #If it didn't work, there is no column at that index. 
                    print('column',column_idx,'not found, end row ~~') #####################################################
                    break

                # next column
                column_idx += 1

            # That's it for the columns in this row.  Store row_data in webpage_data.
            webpage_data.append(row_data)
            
        except:
            # If it didn't work, there is no row at that index.
            print('row',row_idx,'not found, end page ~~~') #####################################################
            break

        # next row
        row_idx += 1

    # That's it for the rows in the table. Done with this webpage!

    print(webpage_data)

    stop_time = datetime.now()
    time_diff = stop_time-start_time
    print('elapsed time:', time_diff.seconds + time_diff.microseconds/1000000, 's')


    myFile = open('example-CHI-{y}-{m}.csv'.format(y=date.year, m=date.month), 'w')
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(webpage_data)

    return webpage_data


chromedriver = '/Users/grazillionaire/Documents/chromedriver' # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)


# Generate list of dates (months) to pull weather data for.

# For Chicago
city = 'il/chicago/KORD'
start_date = datetime(2001,1,1)
end_date = datetime(2018,6,30)

# For New York
#city = 'ny/new-york/KNYC'
#start_date = datetime(2010,5,5)
#end_date = datetime.today()

date_list = pd.Series(pd.date_range(start_date,end_date))
date_list = [datetime.today()] ################# delete this ##################
#city = 'ca/san-francisco/KSFO' ################ delete this ##################




for date in date_list:
    
    url = 'https://www.wunderground.com/history/monthly/us/{loc}/date/{y}-{m}?cm_ven=localwx_history'\
          .format(loc=city, y=date.year, m=date.month)
    
    webpage_data = read_webpage(driver, url)



driver.close()

webpage gotten
header stuff
row idx: 3
column idx: 0
3	subcolumn 1 not found, end column ~
column data [3]
column idx: 1
88.0	74.0	59.0	subcolumn 3 not found, end column ~
column data [88.0, 74.0, 59.0]
column idx: 2
68.0	62.0	56.0	subcolumn 3 not found, end column ~
column data [68.0, 62.0, 56.0]
column idx: 3
93.0	nan	45.0	subcolumn 3 not found, end column ~
column data [93.0, nan, 45.0]
column idx: 4
28.0	nan	6.0	subcolumn 3 not found, end column ~
column data [28.0, nan, 6.0]
column idx: 5
30.05	nan	29.68	subcolumn 3 not found, end column ~
column data [30.05, nan, 29.68]
column idx: 6
nan	0.0	nan	subcolumn 3 not found, end column ~
column data [nan, 0.0, nan]
column idx: 7
column 7 not found, end row ~~
row idx: 4
column idx: 0
4	subcolumn 1 not found, end column ~
column data [4]
column idx: 1
77.0	65.0	52.0	subcolumn 3 not found, end column ~
column data [77.0, 65.0, 52.0]
column idx: 2
69.0	51.0	41.0	subcolumn 3 not found, end column ~
column data [69.0, 51.0, 41.0]
column idx:

In [16]:
column_idx

2

In [17]:
subcolumn_idx

3